In [1]:
import pandas as pd

<h1>Creación de KPIs</h1>

In [2]:
# Cargamos el archivo de hechos (solo las columnas necesarias):
hechos_df = pd.read_csv(r"../Archivos producidos/hechos_homicidios.csv")[["ID", "FECHA", "VICTIMA"]]
hechos_df.head()

,ID,FECHA,VICTIMA
0,2016-0001,2016-01-01,MOTO
1,2016-0002,2016-01-02,AUTO
2,2016-0003,2016-01-03,MOTO
3,2016-0004,2016-01-10,MOTO
4,2016-0005,2016-01-21,MOTO


<h3>KPI 1:</h3>
Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

Definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: (Número de homicidios en siniestros viales / Población total) * 100,000.

In [113]:
# Aseguramos que la columna "FECHA" esté en formato datetime:
hechos_df["FECHA"] = pd.to_datetime(hechos_df["FECHA"])
hechos_df.dtypes

ID                 object
FECHA      datetime64[ns]
VICTIMA            object
dtype: object

In [114]:
# Creamos un nuevo DataFrame para almacenar la cantidad de fatalidades viales por trimestre:
trimestral = pd.DataFrame()
trimestral["TRIMESTRE"] = hechos_df["FECHA"].dt.to_period("Q")

# Contamos las fatalidades por trimestre y fusionamos el resultado con el nuevo DataFrame:
resultados_trimestre = hechos_df.groupby(trimestral["TRIMESTRE"])["ID"].nunique().reset_index()
trimestral = pd.merge(trimestral, resultados_trimestre, on="TRIMESTRE", how="left")

# Renombramos las columnas resultantes:
trimestral.columns = ["TRIMESTRE", "FATALIDADES"]

# Filtramos los registros únicos y reseteamos el índice:
trimestral = trimestral.drop_duplicates().reset_index().drop(columns="index")

trimestral.head()

,TRIMESTRE,FATALIDADES
0,2016Q1,32
1,2016Q2,32
2,2016Q3,35
3,2016Q4,45
4,2017Q1,29


In [115]:
# Creamos un nuevo DataFrame semestral:

# Datos incompletos:
data = {"SEMESTRE": ["2016-1", "2016-2", "2017-1", "2017-2","2018-1", "2018-2", "2019-1", "2019-2","2020-1", "2020-2","2021-1", "2021-2"],
        "FATALIDADES": []}

# Completamos los datos:
for i in [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22]:
    data["FATALIDADES"].append(trimestral.loc[i]["FATALIDADES"] + trimestral.loc[i+1]["FATALIDADES"])

# Guardamos los datos en un DataFrame:
semestral = pd.DataFrame(data)

semestral.head()

,SEMESTRE,FATALIDADES
0,2016-1,64
1,2016-2,80
2,2017-1,64
3,2017-2,67
4,2018-1,67


In [116]:
# Calculamos la tasa de homicidios en siniestros viales:
semestral["TASA"] = (semestral["FATALIDADES"] / 3121707) * 100000

semestral

,SEMESTRE,FATALIDADES,TASA
0,2016-1,64,2.050160
1,2016-2,80,2.562700
2,2017-1,64,2.050160
3,2017-2,67,2.146262
4,2018-1,67,2.146262
5,2018-2,76,2.434565
6,2019-1,56,1.793890
7,2019-2,47,1.505587
8,2020-1,31,0.993046
9,2020-2,47,1.505587


In [117]:
# Calculamos la reducción en porcentaje:
semestral["REDUCCION%"] = semestral["TASA"].pct_change() * 100

semestral


,SEMESTRE,FATALIDADES,TASA,REDUCCION%
0,2016-1,64,2.050160,NaN
1,2016-2,80,2.562700,25.000000
2,2017-1,64,2.050160,-20.000000
3,2017-2,67,2.146262,4.687500
4,2018-1,67,2.146262,0.000000
5,2018-2,76,2.434565,13.432836
6,2019-1,56,1.793890,-26.315789
7,2019-2,47,1.505587,-16.071429
8,2020-1,31,0.993046,-34.042553
9,2020-2,47,1.505587,51.612903


In [104]:
# Guardamos como CSV:
semestral.to_csv(r"../Archivos Producidos/kpi1.csv", index=False)

<h3>KPI 2:</h3>
Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100.

In [118]:
# Nuestro punto de partida será nuevamente el DataFrame de hechos:
hechos_df.head()

,ID,FECHA,VICTIMA
0,2016-0001,2016-01-01,MOTO
1,2016-0002,2016-01-02,AUTO
2,2016-0003,2016-01-03,MOTO
3,2016-0004,2016-01-10,MOTO
4,2016-0005,2016-01-21,MOTO


In [119]:
# Filtramos las fatalidades cuya víctimas circulaban en motocicleta:
motos = hechos_df[hechos_df["VICTIMA"] == "MOTO"].drop(columns="VICTIMA")
motos.head()

,ID,FECHA
0,2016-0001,2016-01-01
2,2016-0003,2016-01-03
3,2016-0004,2016-01-10
4,2016-0005,2016-01-21
5,2016-0008,2016-01-24


In [124]:
# Creamos un nuevo DataFrame para almacenar la cantidad de fatalidades de motos por año:
anual = pd.DataFrame()
anual["AÑO"] = motos["FECHA"].dt.to_period("Y")

# Contamos las fatalidades por año y fusionamos el resultado con el nuevo DataFrame:
resultados_anuales = motos.groupby(anual["AÑO"])["ID"].nunique().reset_index()
anual = pd.merge(anual, resultados_anuales, on="AÑO", how="left")

# Renombramos las columnas resultantes:
anual.columns = ["AÑO", "FATALIDADES"]

# Filtramos los registros únicos y reseteamos el índice:
anual = anual.drop_duplicates().reset_index().drop(columns="index")

anual

,AÑO,FATALIDADES
0,2016,64
1,2017,53
2,2018,55
3,2019,50
4,2020,27
5,2021,46


In [125]:
# Calculamos la reducción en porcentaje:
anual["REDUCCION%"] = anual["FATALIDADES"].pct_change() * 100

anual

,AÑO,FATALIDADES,REDUCCION%
0,2016,64,NaN
1,2017,53,-17.187500
2,2018,55,3.773585
3,2019,50,-9.090909
4,2020,27,-46.000000
5,2021,46,70.370370


In [126]:
# Guardamos como CSV:
anual.to_csv(r"../Archivos Producidos/kpi2.csv", index=False)

<h3>KPI 3:</h3>
Reducir en un 10% la cantidad de accidentes nocturnos fatales de jóvenes en el último semestre, en CABA, respecto al semestre anterior.

Definimos a la cantidad de accidentes nocturnos fatales de jovenes como el número absoluto de siniestros viales fatales, ocurridos entre las 22:00 y las 7:00, en los que estuvieron involucradas víctimas de entre 15 y 29 años de edad, en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes fatales con víctimas en moto es: (Número de accidentes nocturnos fatales de jóvenes en el año anterior - Número de accidentes nocturnos fatales de jóvenes en el año actual) / (Número de accidentes nocturnos fatales de jóvenes en el año anterior) * 100.

In [166]:
# Cargamos el archivo de homicidios:
homicidios_df = pd.read_csv(r"../Archivos producidos/homicidios_fusion.csv")

# Aseguramos que la columna "FECHA" esté en formato datetime:
homicidios_df["FECHA"] = pd.to_datetime(homicidios_df["FECHA"])
homicidios_df.dtypes

ID                         object
N_VICTIMAS                  int64
FECHA              datetime64[ns]
AAAA                        int64
MM                          int64
DD                          int64
HH                        float64
LUGAR_DEL_HECHO            object
TIPO_DE_CALLE              object
VICTIMA                    object
ACUSADO                    object
ROL_VICTIMA                object
SEXO_VICTIMA               object
EDAD_VICTIMA              float64
CALLE                      object
CRUCE                      object
LATITUD                   float64
LONGITUD                  float64
DIA_SEMANA                 object
dtype: object

In [167]:
# Filtramos por edad:
mayor_15 = homicidios_df[homicidios_df["EDAD_VICTIMA"]  >= 15]
y_menor_29 = mayor_15[mayor_15["EDAD_VICTIMA"] <= 29]

# Filtramos por horario:
despues_22 = y_menor_29[y_menor_29["HH"] >= 22]
antes_7 = y_menor_29[y_menor_29["HH"] <= 7]

resultado = pd.concat([despues_22, antes_7], ignore_index=True)
resultado.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,VICTIMA,ACUSADO,ROL_VICTIMA,SEXO_VICTIMA,EDAD_VICTIMA,CALLE,CRUCE,LATITUD,LONGITUD,DIA_SEMANA
0,2016-0016,1,2016-02-14,2016,2,14,22.0,AV REGIMIENTO DE PATRICIOS Y QUINQUELA MARTIN ...,AVENIDA,MOTO,AUTO,PASAJERO_ACOMPAÑANTE,FEMENINO,19.0,REGIMIENTO DE PATRICIOS AV.,"QUINQUELA MARTIN, BENITO",-58.368827,-34.642920,Sunday
1,2016-0102,1,2016-08-10,2016,8,10,23.0,AV EVA PERON Y JOSE LEON SUAREZ,AVENIDA,MOTO,CARGAS,CONDUCTOR,MASCULINO,29.0,"PERON, EVA AV.","SUAREZ, JOSE LEON",-58.500116,-34.673165,Wednesday
2,2017-0041,1,2017-04-03,2017,4,3,23.0,BALDOMERO FERNANDEZ MORENO 2186,CALLE,MOTO,AUTO,CONDUCTOR,MASCULINO,17.0,"FERNANDEZ MORENO, BALDOMERO",NaN,-58.454851,-34.635319,Monday
3,2017-0072,1,2017-06-03,2017,6,3,23.0,AV. DE LOS CONSTITUYENTES Y VALLEJOS,AVENIDA,AUTO,AUTO,CONDUCTOR,MASCULINO,25.0,DE LOS CONSTITUYENTES AV.,VALLEJOS,-58.491751,-34.582270,Saturday
4,2017-0116,1,2017-09-20,2017,9,20,22.0,COLECTORA AV. GRAL. PAZ 5750,GRAL PAZ,MOTO,OBJETO FIJO,CONDUCTOR,MASCULINO,22.0,"PAZ, GRAL. AV.",NaN,-58.513894,-34.580758,Wednesday


In [168]:
juveniles = resultado[["ID", "FECHA"]]
juveniles.head()

,ID,FECHA
0,2016-0016,2016-02-14
1,2016-0102,2016-08-10
2,2017-0041,2017-04-03
3,2017-0072,2017-06-03
4,2017-0116,2017-09-20


In [170]:
# Creamos un nuevo DataFrame para almacenar la cantidad de fatalidades juveniles por trimestre:
trimestral = pd.DataFrame()
trimestral["TRIMESTRE"] = juveniles["FECHA"].dt.to_period("Q")

# Contamos las fatalidades por trimestre y fusionamos el resultado con el nuevo DataFrame:
resultados_trimestre = juveniles.groupby(trimestral["TRIMESTRE"])["ID"].nunique().reset_index()
trimestral = pd.merge(trimestral, resultados_trimestre, on="TRIMESTRE", how="left")

# Renombramos las columnas resultantes:
trimestral.columns = ["TRIMESTRE", "FATALIDADES"]

# Filtramos los registros únicos y reseteamos el índice:
trimestral = trimestral.drop_duplicates().reset_index().drop(columns="index")

trimestral.head()

,TRIMESTRE,FATALIDADES
0,2016Q1,7
1,2016Q3,6
2,2017Q2,4
3,2017Q3,6
4,2017Q4,6


In [173]:
# Creamos un nuevo DataFrame semestral:

# Datos incompletos:
data = {"SEMESTRE": ["2016-1", "2016-2", "2017-1", "2017-2","2018-1", "2018-2", "2019-1", "2019-2","2020-1", "2020-2","2021-1", "2021-2"],
        "FATALIDADES": []}

# Completamos los datos:
for i in [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22]:
    data["FATALIDADES"].append(trimestral.loc[i]["FATALIDADES"] + trimestral.loc[i+1]["FATALIDADES"])

# Guardamos los datos en un DataFrame:
semestral = pd.DataFrame(data)

semestral.head()

,SEMESTRE,FATALIDADES
0,2016-1,13
1,2016-2,10
2,2017-1,14
3,2017-2,10
4,2018-1,9


In [174]:
# Calculamos la reducción en porcentaje:
semestral["REDUCCION%"] = semestral["FATALIDADES"].pct_change() * 100

semestral

,SEMESTRE,FATALIDADES,REDUCCION%
0,2016-1,13,NaN
1,2016-2,10,-23.076923
2,2017-1,14,40.000000
3,2017-2,10,-28.571429
4,2018-1,9,-10.000000
5,2018-2,4,-55.555556
6,2019-1,14,250.000000
7,2019-2,4,-71.428571
8,2020-1,17,325.000000
9,2020-2,7,-58.823529


In [176]:
# Guardamos como CSV:
semestral.to_csv(r"../Archivos Producidos/kpi3.csv", index=False)